In [114]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# set standard size for plots
plt.rcParams["figure.figsize"] = (8,5)

In [115]:
PATH = "Indian_Kids_Screen_Time.csv"

# reads file so that none is a legit answer and not considered missing
df = pd.read_csv(PATH, na_filter=False)

# general data exploration
df.head()
df.info()
df.describe()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9712 entries, 0 to 9711
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                9712 non-null   int64  
 1   Gender                             9712 non-null   object 
 2   Avg_Daily_Screen_Time_hr           9712 non-null   float64
 3   Primary_Device                     9712 non-null   object 
 4   Exceeded_Recommended_Limit         9712 non-null   bool   
 5   Educational_to_Recreational_Ratio  9712 non-null   float64
 6   Health_Impacts                     9712 non-null   object 
 7   Urban_or_Rural                     9712 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(4)
memory usage: 540.7+ KB


,0
Age,0
Gender,0
Avg_Daily_Screen_Time_hr,0
Primary_Device,0
Exceeded_Recommended_Limit,0
Educational_to_Recreational_Ratio,0
Health_Impacts,0
Urban_or_Rural,0


In [116]:
# Rename columns to simpler names
df = df.rename(columns={
    "Exceeded_Recommended_Limit": "exceed",
    "Avg_Daily_Screen_Time_hr": "screentime",
    "Educational_to_Recreational_Ratio": "edu_rec_ratio",
    "Urban_or_Rural": "urban_rural",
    "Primary_Device": "device",
    "Health_Impacts": "impacts",
    "Gender": "gender"
})

# Make target numeric- 0/1
df["exceed"] = df["exceed"].astype(bool).astype(int)


# categorical encoding - internally one hot encodes these variables
categories = ["gender","device","impacts","urban_rural"]
for c in categories:
    df[c] = df[c].astype("category")


In [117]:
# outcome distribution - gives proportions
df["exceed"].value_counts(normalize=True).rename({0:"No",1:"Yes"})

# check outcome distribution and explore groups
df.groupby("device")["screentime"].mean().sort_values()
df.groupby("urban_rural")["screentime"].mean()
df.groupby("exceed")["screentime"].mean()


/tmp/ipython-input-2259712462.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("device")["screentime"].mean().sort_values()
/tmp/ipython-input-2259712462.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("urban_rural")["screentime"].mean()


,screentime
exceed,
0,1.581751
1,4.823865


In [118]:
# chi squared check if exceeding screentime limit is associated with certain devices
ct = pd.crosstab(df["device"], df["exceed"])
chi2, p, dof, exp = stats.chi2_contingency(ct)
# obvserved results
print("Chi-square:", chi2, "p-value:", p, "dof:", dof)
# expected results if null hypothesis is true (independence)
ct, pd.DataFrame(exp, index=ct.index, columns=ct.columns)

Chi-square: 42.33917841275176 p-value: 3.3993201695284536e-09 dof: 3


(exceed        0     1
 device               
 Laptop      158  1275
 Smartphone  619  3949
 TV          402  2085
 Tablet      232   992,
 exceed               0            1
 device                             
 Laptop      208.192236  1224.807764
 Smartphone  663.658155  3904.341845
 TV          361.321767  2125.678233
 Tablet      177.827842  1046.172158)

In [119]:
# Cramer's shows how significant the relationship is from chi squared
chi2, p, dof, exp = stats.chi2_contingency(ct)
n = ct.to_numpy().sum()
r, k = ct.shape
V = np.sqrt(chi2 / (n*(min(r,k)-1)))
print("Cramér’s V:", V)

Cramér’s V: 0.06602628714681535


In [120]:
x = df.loc[df["exceed"]==1, "screentime"]
y = df.loc[df["exceed"]==0, "screentime"]

# Welch checks if mean screentime of kids who exceed limit is very different from those who dont
t, p_t = stats.ttest_ind(x, y, equal_var=False)
# Mann–Whitney U to check the same
u, p_u = stats.mannwhitneyu(x, y, alternative="two-sided")

print(f"t-test: t={t:.3f}, p={p_t:.3g}")
print(f"Mann–Whitney U: U={u:.0f}, p={p_u:.3g}")
x.mean(), y.mean()


t-test: t=104.630, p=0
Mann–Whitney U: U=11601230, p=0


(np.float64(4.823864594627152), np.float64(1.5817505315379166))

In [121]:
x = df.loc[df.exceed==1, "screentime"]
y = df.loc[df.exceed==0, "screentime"]
nx, ny = x.size, y.size

# differences in mean and variance
mean_diff = x.mean() - y.mean()
sx2, sy2 = x.var(ddof=1), y.var(ddof=1)

# Welch's and 95% confidence interval
se = np.sqrt(sx2/nx + sy2/ny)
ci_low, ci_high = mean_diff + stats.t.ppf([0.025, 0.975], df=min(nx-1,ny-1))*se
pooled_sd = np.sqrt(((nx-1)*sx2 + (ny-1)*sy2) / (nx+ny-2))

# Cohen's to standardize effect size
cohens_d = mean_diff / pooled_sd
mean_diff, (ci_low, ci_high), cohens_d


(np.float64(3.242114063089236),
 (np.float64(3.181329417695918), np.float64(3.302898708482554)),
 np.float64(2.5262040431553747))

In [122]:
from statsmodels.stats.power import TTestIndPower
analysis = TTestIndPower()
d_est = cohens_d

# tests power of two sided t test
power = analysis.power(effect_size=d_est, nobs1=x.size, ratio=y.size/x.size, alpha=0.05, alternative='two-sided')
power


np.float64(1.0)

In [123]:
# One-way ANOVA using statsmodels
model_anova = smf.ols("screentime ~ C(device)", data=df).fit()
anova_table = sm.stats.anova_lm(model_anova, typ=2)
anova_table


,sum_sq,df,F,PR(>F)
C(device),52.350725,3.0,5.919665,0.000497
Residual,28617.655824,9708.0,NaN,NaN


In [124]:
ss_between = anova_table.loc["C(device)", "sum_sq"]
ss_within  = anova_table.loc["Residual", "sum_sq"]
df_between = anova_table.loc["C(device)", "df"]
df_within  = anova_table.loc["Residual", "df"]

# effect sizes for ANOVA
eta_sq = ss_between / (ss_between + ss_within)
omega_sq = (ss_between - df_between*(ss_within/df_within)) / (ss_between + ss_within + (ss_within/df_within))
eta_sq, omega_sq


(np.float64(0.0018259753334767647), np.float64(0.0015173601023848989))

In [125]:
# Pearson correlation to cjeck age vs screentime correlation
r, p = stats.pearsonr(df["Age"], df["screentime"])
print(f"Pearson r={r:.3f}, p={p:.3g}")

# Spearman correlation to check the same
rho, p_s = stats.spearmanr(df["Age"], df["screentime"])
print(f"Spearman rho={rho:.3f}, p={p_s:.3g}")


Pearson r=0.118, p=1.26e-31
Spearman rho=0.105, p=2.33e-25


In [126]:
r, _ = stats.pearsonr(df["Age"], df["screentime"])
n = len(df)
z = np.arctanh(r); se = 1/np.sqrt(n-3)
z_ci = z + stats.norm.ppf([0.025,0.975])*se
r_ci = np.tanh(z_ci); (r, r_ci[0], r_ci[1])

(np.float64(0.11832766082818066),
 np.float64(0.09867130025459939),
 np.float64(0.1378917213171308))

In [127]:
# logistic regression
formula = (
    "exceed ~ Age + edu_rec_ratio "
    "+ C(gender) + C(urban_rural) + C(device)"
)
logit = smf.logit(formula, data=df).fit()
print(logit.summary())

# odds ratios with 95% CIs
params = logit.params
conf   = logit.conf_int()
or_table = pd.DataFrame({
    "odds_ratio": np.exp(params),
    "ci_lower":   np.exp(conf[0]),
    "ci_upper":   np.exp(conf[1]),
})
or_table.sort_values("odds_ratio", ascending=False).round(3)


Optimization terminated successfully.
         Current function value: 0.399521
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 exceed   No. Observations:                 9712
Model:                          Logit   Df Residuals:                     9704
Method:                           MLE   Df Model:                            7
Date:                Sun, 24 Aug 2025   Pseudo R-squ.:                 0.03600
Time:                        20:18:00   Log-Likelihood:                -3880.1
converged:                       True   LL-Null:                       -4025.1
Covariance Type:            nonrobust   LLR p-value:                 9.057e-59
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   1.2215      0.314      3.894      0.000       0.607   

,odds_ratio,ci_lower,ci_upper
Intercept,3.392,1.834,6.274
C(gender)[T.Male],1.200,1.070,1.346
Age,1.123,1.098,1.148
C(urban_rural)[T.Urban],1.002,0.884,1.136
C(device)[T.Smartphone],0.988,0.817,1.195
C(device)[T.TV],0.947,0.770,1.164
C(device)[T.Tablet],0.798,0.635,1.003
edu_rec_ratio,0.117,0.048,0.288


In [128]:
# GLRT checks- does adding predictors to only age model improve fit?
m_base = smf.logit("exceed ~ Age", data=df).fit(disp=False)
m_full = logit  # from earlier

LR = 2 * (m_full.llf - m_base.llf)
df_diff = m_full.df_model - m_base.df_model
p_lr = stats.chi2.sf(LR, df_diff)
LR, df_diff, p_lr


(np.float64(39.22490713845036), 6.0, np.float64(6.46651064152487e-07))